In [9]:
### Question 1.3.2

# import packages
import numpy as np

# calculate fixed point conditions
def calc_fp(p, α, δ, x):
    f1 = np.exp(x*α - δ*p[1])/(1+np.exp(x*α - δ*p[1])) - p[0]
    f2 = np.exp(x*α - δ*p[0])/(1+np.exp(x*α - δ*p[0])) - p[1]
    return np.array([f1, f2])

# calculate jacobian
def calc_jac(p, α, δ, x):
    j11 = -1
    j12 = -δ*(np.exp(x*α - δ*p[1])/(1+np.exp(x*α - δ*p[1])))/(1+np.exp(x*α - δ*p[1]))
    j21 = -δ*(np.exp(x*α - δ*p[0])/(1+np.exp(x*α - δ*p[0])))/(1+np.exp(x*α - δ*p[0]))
    j22 = -1
    j = np.array((j11, j12, j21, j22), dtype=float).reshape(2,2)
    return j

# solve using newton's method
def newton_solver(p, α, δ, x):
    counter = 1
    diff = 100
    tol = 1e-14
    maxiter = 10000
    p_next = p.reshape((2,1))
    while (diff > tol and counter < maxiter):
        p = p_next
        f = calc_fp(p, α, δ, x)
        j = calc_jac(p, α, δ, x)
        p_next = p - np.linalg.inv(j)@f
        diff = np.max(np.abs(p_next - p))
        counter += 1
    return p_next.flatten()

# search over grid for multiple equilibria
def find_bne(α, δ, x, sym=False):
    looper = lambda p_guess: newton_solver(p_guess, α, δ, x)
    grid =  np.mgrid[0:1:1e-1, 0:1:1e-1].reshape(2,-1).T
    sols = np.round(np.apply_along_axis(looper, 1, grid),6)
    eqs = np.unique(sols, axis=0)
    if sym == True:
        return eqs[np.argmin(np.abs(eqs[:,0] - eqs[:,1])), :]    
    return eqs

# results
find_bne(1,1,1)
find_bne(1,1,2)
find_bne(3,6,1)
find_bne(3,6,2)

array([[0.71179 , 0.849318],
       [0.784577, 0.784577],
       [0.849318, 0.71179 ]])

In [24]:
### Question 1.3.3

# set up
T = 1000
S = 50
α_set = 3
δ_set = 6

# simulate data
def sim_sample(α,δ,T):
    ε1 = np.random.logistic(0, 1, T*S).reshape(T,S)
    ε2 = np.random.logistic(0, 1, T*S).reshape(T,S)
    x = (np.random.binomial(1,.5, T*S) + 1).reshape(T,S)
    return ε1,ε2,x

ε1_sim,ε2_sim,x_sim = sim_sample(3,6,T)

# probabilities for each x
def find_prob(α, δ, sym=False):
    p1,p2 = find_bne(α, δ, 1, sym=sym),find_bne(α, δ, 2, sym=sym)
    return p1, p2

# symmetric probabilities 
p1_sim, p2_sim = find_prob(3,6,sym=True)
probs_sim = np.zeros((T,S))
probs_sim[x_sim == 1] = p1_sim[0]
probs_sim[x_sim == 2] = p2_sim[0]
print(p1_sim, p2_sim)

# y1, y2
y1_sim = np.array(1*(-δ_set*probs_sim + α_set*x_sim + ε1_sim >= 0))
y2_sim = np.array(1*(-δ_set*probs_sim + α_set*x_sim + ε2_sim >= 0))

# log likelihood function
def sym_lik(ω, y1, y2, x):
    # parse ω
    α = ω[0]
    δ = ω[1]
    # calculating p for each x value
    probs = np.zeros((x.size))
    p_x1, p_x2 = findps(α, δ, sym=True)
    probs[x==1] = p_x1[0]
    probs[x==2] = p_x2[0]
    # likelihood per market
    ll1 = y1*np.log(np.exp(δ*probs - α*x)/(1+np.exp(δ*probs - α*x))) + (1-y1)*np.log(1/(1+np.exp(δ*probs - α*x )))
    ll2 = y2*np.log(np.exp(δ*probs - α*x)/(1+np.exp(δ*probs - α*x))) + (1-y2)*np.log(1/(1+np.exp(δ*probs - α*x )))
    ll = ll1 + ll2  
    return -np.sum(ll)

# how to optimize?

[0.5 0.5] [0.784577 0.784577]
